In [9]:
# Enable auto-reloading of external modules - useful during development
%load_ext autoreload
%autoreload 2

# Configure Python path to find our custom modules
import sys
from pathlib import Path

# Add project root to the Python path for proper imports
project_root = Path.cwd().parent
if project_root not in sys.path:
    sys.path.insert(0, str(project_root))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Import necessary libraries
import src.processing as processing
import src.config as lists

In [11]:
# Load and process data
df = processing.load_data("/Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/data/raw/nvzfxcoxdvh1at7i.csv")
df_prepared = processing.prepare_data(df)
df_added_features = processing.create_all_model_features_orchestrated(df_prepared)
df_missing = processing.drop_missing_final_vars_streamlined(df_added_features, lists.final_set_A_predictor_names_and_dependent)
df_final = processing.annual_winsorize_variables(df_missing, lists.columns_to_winsorize)

/Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/src/processing.py:41: DtypeWarning: Columns (10,11,12,16,17,22,26,30,33,34,35,945,946,948,949,950,955,969,970,975,980) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


Data loaded successfully from /Users/luis.m/Library/Mobile Documents/com~apple~CloudDocs/Documents ☁️/VSC Projects/Master_Thesis/data/raw/nvzfxcoxdvh1at7i.csv
Original number of observations: 317304
Number of columns after selection: 30
Observations after year filter (2000-2023): 302751
Observations after excluding financial and utility firms: 170598
Starting feature construction. Initial df shape: (170598, 30)
  Creating lags for: ['at', 'ni', 'rect', 'invt', 'ap', 'sale']

Performing pre-calculation validity checks & preparations...
  Missing 'xrd' values filled with 0.
  'ipo_year' created from 'ipodate'.

Constructing dependent variable...
  OCF_Scaled_t_plus_1 created.

Constructing Set A (OLS) predictors...
  Set A predictors constructed.

Constructing control dummy variables...
  Dummy variables constructed.

Constructing Set B (additional ML) predictors...
  Set B predictors constructed.

Selecting final model variables and dropping intermediate columns...
  Shape of DataFrame 

In [12]:
# Split data chronologically - train on pre-2018, test on 2018+
train_df, test_df = processing.split_data_chronologically(df_final, 'fyear', split_year=2018)

print(f"Training data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")
print(f"Training period: {train_df['fyear'].min()} - {train_df['fyear'].max()}")
print(f"Test period: {test_df['fyear'].min()} - {test_df['fyear'].max()}")

Training set: 102567 obs (Predictor years <= 2018)
Test set: 20882 obs (Predictor years > 2018)
Training data shape: (102567, 31)
Test data shape: (20882, 31)
Training period: 2001.0 - 2018.0
Test period: 2019.0 - 2022.0


In [13]:
# Prepare features and dependent variable for XGBoost regression
X_train = train_df[lists.SET_A_FEATURES]
y_train = train_df[lists.DEPENDENT_VARIABLE]

X_test = test_df[lists.SET_A_FEATURES]
y_test = test_df[lists.DEPENDENT_VARIABLE]

print("Features included in the model:")
for i, feature in enumerate(X_train.columns, 1):
    print(f"{i:2d}. {feature}")

print(f"\nDependent variable: {lists.DEPENDENT_VARIABLE}")
print(f"Training observations: {len(X_train)}")
print(f"Test observations: {len(X_test)}")

Features included in the model:
 1. OCF_Scaled_Lag_t
 2. NI_Scaled_t
 3. Accruals_Scaled_t
 4. Delta_Rec_Scaled_t
 5. Delta_Inv_Scaled_t
 6. Delta_AP_Scaled_t
 7. DP_Scaled_t
 8. ln_at_t

Dependent variable: OCF_Scaled_t_plus_1
Training observations: 102567
Test observations: 20882


In [14]:
# =============================================================================
# BAYESIAN OPTIMIZATION FOR HYPERPARAMETER TUNING
# =============================================================================

import xgboost as xgb
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import numpy as np
import time

# Define hyperparameter search space for Bayesian optimization
search_space = {
    'max_depth': Integer(3, 10),
    'learning_rate': Real(0.01, 0.2, prior='log-uniform'),
    'n_estimators': Integer(100, 300),
    'subsample': Real(0.8, 1.0),
    'colsample_bytree': Real(0.8, 1.0)
}

# Create XGBoost regressor
xgb_regressor = xgb.XGBRegressor(random_state=42, objective='reg:squarederror')

# Perform Bayesian Search with 5-fold cross-validation
print("Performing 5-fold cross-validation for XGBoost Bayesian hyperparameter tuning...")
print(f"Search space dimensions: {len(search_space)}")
print(f"Number of iterations: 100 (intelligent sampling)")

start_time = time.time()

bayes_search = BayesSearchCV(
    estimator=xgb_regressor,
    search_spaces=search_space,
    n_iter=100,  # Number of parameter settings to try
    cv=5,
    scoring='r2',
    n_jobs=-1,
    random_state=42,
    verbose=2  # Increased verbosity for more detailed progress information
)

# Fit the bayesian search
bayes_search.fit(X_train, y_train)

# Print best parameters and score
print(f"\nBest hyperparameters: {bayes_search.best_params_}")
print(f"Best cross-validation R² score: {bayes_search.best_score_:.4f}")
print(f"Standard deviation: {bayes_search.cv_results_['std_test_score'][bayes_search.best_index_]:.4f}")

elapsed_time = time.time() - start_time
print(f"Bayesian search completed in {elapsed_time:.2f} seconds")

# For compatibility with rest of the code, assign to grid_search variable
grid_search = bayes_search

Performing 5-fold cross-validation for XGBoost Bayesian hyperparameter tuning...
Search space dimensions: 5
Number of iterations: 100 (intelligent sampling)
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END colsample_bytree=0.8820207917706628, learning_rate=0.08846938749167613, max_depth=10, n_estimators=163, subsample=0.9340295896537869; total time=   2.0s
[CV] END colsample_bytree=0.8820207917706628, learning_rate=0.08846938749167613, max_depth=10, n_estimators=163, subsample=0.9340295896537869; total time=   2.0s
[CV] END colsample_bytree=0.8820207917706628, learning_rate=0.08846938749167613, max_depth=10, n_estimators=163, subsample=0.9340295896537869; total time=   2.0s
[CV] END colsample_bytree=0.8820207917706628, learning_rate=0.08846938749167613, max_depth=10, n_estimators=163, subsample=0.9340295896537869; total time=   2.0s
[CV] END colsample_bytree=0.8820207917706628, learning_rate=0.08846938749167613, max_depth=10, n_estimators=163, subsample=0.93402958965

In [ ]:
# =============================================================================
# XGBOOST MODEL - For ML Model Comparisons
# =============================================================================

# Use the best model from grid search
best_xgb_model = grid_search.best_estimator_

# Generate predictions using the optimized XGBoost model
y_train_pred = best_xgb_model.predict(X_train)
y_test_pred = best_xgb_model.predict(X_test)

# Performance evaluation function for model metrics
def print_model_performance(y_true, y_pred, model_name, dataset):
    """
    Calculate and display performance metrics for regression models.
    
    Parameters:
    - y_true: actual values
    - y_pred: predicted values
    - model_name: model name for display
    - dataset: dataset name (Train/Test)
    """
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    print(f"{model_name} ({dataset}): R²={r2:.4f} | RMSE={rmse:.4f} | MAE={mae:.4f}")

# Display XGBoost performance results
print("XGBOOST PERFORMANCE:")
print_model_performance(y_train, y_train_pred, "XGBoost", "Train")
print_model_performance(y_test, y_test_pred, "XGBoost", "Test")

XGBOOST PERFORMANCE:
XGBoost (Train): R²=0.7265 | RMSE=0.5542 | MAE=0.1819
XGBoost (Test): R²=0.6030 | RMSE=0.6145 | MAE=0.2082


In [16]:
# =============================================================================
# FEATURE IMPORTANCE ANALYSIS
# =============================================================================

import pandas as pd

# Get feature importances
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("FEATURE IMPORTANCE RANKING:")
print("="*50)
for i, (_, row) in enumerate(feature_importance.iterrows(), 1):
    print(f"{i:2d}. {row['feature']:<25} {row['importance']:.4f}")

# Print model details
print(f"\nMODEL DETAILS:")
print(f"Max Depth: {best_xgb_model.max_depth}")
print(f"Learning Rate: {best_xgb_model.learning_rate}")
print(f"N Estimators: {best_xgb_model.n_estimators}")
print(f"Subsample: {best_xgb_model.subsample}")
print(f"Colsample Bytree: {best_xgb_model.colsample_bytree}")
print(f"Number of Features: {best_xgb_model.n_features_in_}")

FEATURE IMPORTANCE RANKING:
 1. ln_at_t                   0.5962
 2. OCF_Scaled_Lag_t          0.1850
 3. NI_Scaled_t               0.0508
 4. Delta_Rec_Scaled_t        0.0375
 5. Accruals_Scaled_t         0.0353
 6. Delta_Inv_Scaled_t        0.0318
 7. DP_Scaled_t               0.0318
 8. Delta_AP_Scaled_t         0.0315

MODEL DETAILS:
Max Depth: 6
Learning Rate: 0.023150093482357005
N Estimators: 171
Subsample: 0.8
Colsample Bytree: 1.0
Number of Features: 8


## Why `ln_at_t` (Log of Total Assets)?

### **Academic and Statistical Rationale:**

**1. Distribution Normalization:**
- **Raw Total Assets**: Highly right-skewed (few very large firms, many smaller ones)
- **Log Assets**: More normal distribution, better for statistical modeling
- **Reduces impact of extreme outliers** (mega-corporations vs small firms)

**2. Economic Interpretation:**
- **ln_at_t** captures **relative size differences** rather than absolute differences
- A unit change in log assets represents a **percentage change** in firm size
- More economically meaningful: difference between $1B and $10B firm is similar to difference between $100M and $1B firm

**3. Model Performance:**
- **Linear models (OLS)**: Assume linear relationships - log transformation helps
- **Tree models (XGB/RF)**: Can handle non-linear relationships but still benefit from reduced skewness
- **Prevents large firms from dominating** the model due to scale alone

**4. Standard Practice:**
- **Universal in finance literature** - almost all studies use log assets
- **Comparability** with prior research and academic standards
- **Size control variable** - captures firm size effects on cash flows

**5. Mathematical Benefits:**
- **Reduces heteroscedasticity** (variance changes with firm size)
- **Stabilizes variance** across different firm sizes
- **Better coefficient interpretation** in regression models

### **Alternative Approaches (Not Used):**
- **Raw Assets**: Too skewed, dominated by outliers
- **Scaled Assets**: Would require another scaling variable
- **Size Quintiles**: Loses continuous information
- **Square Root**: Less standard, harder to interpret

**Bottom Line:** `ln_at_t` is the gold standard for controlling firm size in financial models - it's both statistically sound and academically expected.